In [2]:
import os

In [3]:
%pwd

'c:\\Users\\Vaibhav\\Desktop\\Course\\12_project_1\\Project_01\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\Vaibhav\\Desktop\\Course\\12_project_1\\Project_01'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from src.datascience_project.constants import *
from src.datascience_project.utils.common import read_yaml, create_directory

In [8]:
# Updating configuration manager in src file

class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 param_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(param_filepath)
        self.schema=read_yaml(schema_filepath)
        
        create_directory([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directory([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [9]:
import os
import urllib.request as requests
from src.datascience_project import logger
import zipfile

In [10]:
# Update the components

class DataIngestion:
    def __init__(self, config) -> DataIngestionConfig:
        self.config=config

    def downloda_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = requests.urlretrieve(
                url=self.config.source_URL,
                filename= self.config.local_data_file
            ) 
            logger.info(f"{filename} downloaded! {headers}")
        else:
            logger.info(f"File already exits")

    def extract_zipfile(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [11]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.downloda_file()
    data_ingestion.extract_zipfile()
except Exception as e:
    raise e


[2025-05-01 16:28:23,077: INFO: common: yaml file config\config.yaml loaded successfully]
[2025-05-01 16:28:23,080: INFO: common: yaml file params.yaml loaded successfully]
[2025-05-01 16:28:23,083: INFO: common: yaml file schema.yaml loaded successfully]
[2025-05-01 16:28:23,085: INFO: common: created directory at: artifacts]
[2025-05-01 16:28:23,087: INFO: common: created directory at: artifacts/data_ingestion]
[2025-05-01 16:28:24,009: INFO: 2924033922: artifacts/data_ingestion/data.zip downloaded! Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4E15:2749F9:120383:3080E2:681352A1
Accept-Ranges: bytes
Date: Thu, 01 May 2025 10:58:24 GMT
